# Natural Language Processing </a>

## Assignment: K Nearest Neighbors Model for the IMDB Movie Review Dataset

For the final project, build a K Nearest Neighbors model to predict the sentiment (positive or negative) of movie reviews. The dataset is originally hosted here: http://ai.stanford.edu/~amaas/data/sentiment/

Use the notebooks from the class and implement the model, train and test with the corresponding datasets.

You can follow these steps:
1. Read training-test data (Given)
2. Train a KNN classifier (Implement)
3. Make predictions on your test dataset (Implement)

__You can use the KNN Classifier from here: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html__

## 1. Reading the dataset

We will use the __pandas__ library to read our dataset.

#### __Training data:__
Let's read our training data. Here, we have the text and label fields. Labe is 1 for positive reviews and 0 for negative reviews.

In [ ]:
import pandas as pd

train_df = pd.read_csv('https://raw.githubusercontent.com/aws-samples/aws-machine-learning-university-accelerated-nlp/master/data/final_project/imdb_train.csv', header=0)
train_df.head()

,text,label
0,This movie makes me want to throw up every tim...,0
1,Listening to the director's commentary confirm...,0
2,One of the best Tarzan films is also one of it...,1
3,Valentine is now one of my favorite slasher fi...,1
4,No mention if Ann Rivers Siddons adapted the m...,0


#### __Test data:__

In [ ]:
import pandas as pd

test_df = pd.read_csv('https://raw.githubusercontent.com/aws-samples/aws-machine-learning-university-accelerated-nlp/master/data/final_project/imdb_test.csv', header=0)
test_df.head()

,text,label
0,What I hoped for (or even expected) was the we...,0
1,Garden State must rate amongst the most contri...,0
2,There is a lot wrong with this film. I will no...,1
3,"To qualify my use of ""realistic"" in the summar...",1
4,Dirty War is absolutely one of the best politi...,1


## 2. Train a KNN Classifier
Here, you will apply pre-processing operations we covered in the class. Then, you can split your dataset to training and validation here. For your first submission, you will use __K Nearest Neighbors Classifier__. It is available [here](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html).

In [ ]:
# Install the library and functions
import nltk

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import nltk, re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

# Let's get a list of stop words from the NLTK library
stop = stopwords.words('english')

# These words are important for our problem. We don't want to remove them.
excluding = ['against', 'not', 'don', "don't",'ain', 'aren', "aren't", 'couldn', "couldn't",
             'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't",
             'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't",
             'needn', "needn't",'shouldn', "shouldn't", 'wasn', "wasn't", 'weren',
             "weren't", 'won', "won't", 'wouldn', "wouldn't"]

# New stop word list
stop_words = [word for word in stop if word not in excluding]

snow = SnowballStemmer('english')

def process_text(texts):
    final_text_list=[]
    for sent in texts:

        # Check if the sentence is a missing value
        if isinstance(sent, str) == False:
            sent = ""

        filtered_sentence=[]

        sent = sent.lower() # Lowercase
        sent = sent.strip() # Remove leading/trailing whitespace
        sent = re.sub('\s+', ' ', sent) # Remove extra space and tabs
        sent = re.compile('<.*?>').sub('', sent) # Remove HTML tags/markups:

        for w in word_tokenize(sent):
            # We are applying some custom filtering here, feel free to try different things
            # Check if it is not numeric and its length>2 and not in stop words
            if(not w.isnumeric()) and (len(w)>2) and (w not in stop_words):
                # Stem and add to filtered list
                filtered_sentence.append(snow.stem(w))
        final_string = " ".join(filtered_sentence) #final string of cleaned words

        final_text_list.append(final_string)

    return final_text_list

In [ ]:
from sklearn.model_selection import train_test_split

X_train=train_df[["text"]]
X_val=test_df[["text"]]
y_train=train_df["label"]
y_val=test_df["label"]

In [ ]:
print("Processing the reviewText fields")
train_text_list = process_text(X_train["text"].tolist())
val_text_list = process_text(X_val["text"].tolist())

Processing the reviewText fields


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier

import gensim
from gensim.models import Word2Vec
### PIPELINE ###
##########################
w2v = gensim.models.Word2Vec()
pipeline = Pipeline([
    ('text_vect', CountVectorizer(binary=False,
                                  max_features=100)),
    ('knn', KNeighborsClassifier(n_neighbors=15))
                                ])


# Visualize the pipeline
# This will come in handy especially when building more complex pipelines, stringing together multiple preprocessing steps
from sklearn import set_config
set_config(display='diagram')
pipeline

Pipeline(steps=[('text_vect', CountVectorizer(max_features=100)),
                ('knn', KNeighborsClassifier(n_neighbors=15))])

## 3. Make predictions on your test dataset

Once we select our best performing model, we can use it to make predictions on the test dataset. You can simply use __.fit()__ function with your training data to use the best performing K value and use __.predict()__ with your test data to get your test predictions.

In [ ]:
# Implement this

# We using lists of processed text fields
X_train = train_text_list
X_val = val_text_list

# Fit the Pipeline to training data
pipeline.fit(X_train, y_train.values)

Pipeline(steps=[('text_vect', CountVectorizer(max_features=100)),
                ('knn', KNeighborsClassifier(n_neighbors=15))])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Use the fitted pipeline to make predictions on the validation dataset
val_predictions = pipeline.predict(X_val)
print(confusion_matrix(y_val.values, val_predictions))
print(classification_report(y_val.values, val_predictions))
print("Accuracy (validation):", accuracy_score(y_val.values, val_predictions))

[[8127 4373]
 [3979 8521]]
              precision    recall  f1-score   support

           0       0.67      0.65      0.66     12500
           1       0.66      0.68      0.67     12500

    accuracy                           0.67     25000
   macro avg       0.67      0.67      0.67     25000
weighted avg       0.67      0.67      0.67     25000

Accuracy (validation): 0.66592


Binary | Max Features | KNN (n_neighbors) | precision (macro avg) | recall (macro avg) | f1-score (macro avg) | precision (weighted avg) | recall (weighted avg) | f1-score (weighted avg) | Accuracy
------|-----|----|------|------|------|------|------|------|--------
True  | 5   | 3  | 0.50 | 0.50 | 0.49 | 0.50 | 0.50 | 0.49 | 0.49932
True  | 10  | 3  | 0.52 | 0.52 | 0.52 | 0.52 | 0.52 | 0.52 | 0.51768
True  | 25  | 3  | 0.56 | 0.56 | 0.56 | 0.56 | 0.56 | 0.56 | 0.55660
True  | 75  | 3  | 0.62 | 0.62 | 0.62 | 0.62 | 0.62 | 0.62 | 0.62376
True  | 100 | 3  | 0.63 | 0.63 | 0.63 | 0.63 | 0.63 | 0.63 | 0.63080
True  | 5   | 5  | 0.52 | 0.52 | 0.50 | 0.52 | 0.52 | 0.50 | 0.52140
True  | 10  | 5  | 0.52 | 0.52 | 0.52 | 0.52 | 0.52 | 0.52 | 0.52288
True  | 25  | 5  | 0.56 | 0.56 | 0.56 | 0.56 | 0.56 | 0.56 | 0.56472
True  | 75  | 5  | 0.64 | 0.64 | 0.64 | 0.64 | 0.64 | 0.64 | 0.63856
True  | 100 | 5  | 0.64 | 0.64 | 0.64 | 0.64 | 0.64 | 0.64 | 0.63816
True  | 5   | 9  | 0.53 | 0.53 | 0.53 | 0.53 | 0.53 | 0.53 | 0.53012
True  | 10  | 9  | 0.53 | 0.53 | 0.53 | 0.53 | 0.53 | 0.53 | 0.52932
True  | 25  | 9  | 0.57 | 0.57 | 0.57 | 0.57 | 0.57 | 0.57 | 0.57116
True  | 75  | 9  | 0.66 | 0.66 | 0.66 | 0.66 | 0.66 | 0.66 | 0.65572
True  | 100 | 9  | 0.66 | 0.66 | 0.66 | 0.66 | 0.66 | 0.66 | 0.66056
False | 5   | 3  | 0.51 | 0.51 | 0.51 | 0.51 | 0.51 | 0.51 | 0.50944
False | 10  | 3  | 0.52 | 0.52 | 0.52 | 0.52 | 0.52 | 0.52 | 0.52076
False | 25  | 3  | 0.59 | 0.59 | 0.59 | 0.59 | 0.59 | 0.59 | 0.59288
False | 75  | 3  | 0.62 | 0.62 | 0.62 | 0.62 | 0.62 | 0.62 | 0.61512
False | 100 | 3  | 0.62 | 0.62 | 0.62 | 0.62 | 0.62 | 0.62 | 0.61780
False | 5   | 5  | 0.52 | 0.52 | 0.52 | 0.52 | 0.52 | 0.52 | 0.51916
False | 10  | 5  | 0.52 | 0.52 | 0.52 | 0.52 | 0.52 | 0.52 | 0.52424
False | 25  | 5  | 0.60 | 0.60 | 0.60 | 0.60 | 0.60 | 0.60 | 0.60444
False | 75  | 5  | 0.63 | 0.63 | 0.63 | 0.63 | 0.63 | 0.63 | 0.63128
False | 100 | 5  | 0.64 | 0.64 | 0.64 | 0.64 | 0.64 | 0.64 | 0.63772
False | 5   | 9  | 0.53 | 0.53 | 0.53 | 0.53 | 0.53 | 0.53 | 0.52672
False | 10  | 9  | 0.53 | 0.53 | 0.53 | 0.53 | 0.53 | 0.53 | 0.53124
False | 25  | 9  | 0.62 | 0.62 | 0.61 | 0.62 | 0.62 | 0.61 | 0.61512
False | 75  | 9  | 0.65 | 0.65 | 0.65 | 0.65 | 0.65 | 0.65 | 0.64720
False | 100 | 9  | 0.66 | 0.66 | 0.66 | 0.66 | 0.66 | 0.66 | 0.65624
False | 100 | 15 | 0.67 | 0.67 | 0.67 | 0.67 | 0.67 | 0.67 | 0.66592